* PINECONE_API_KEY = 'pcsk_56YtYx_7aMvxTEVd2hcczt7BbyfsUXJ5UvK8hFfP544zJAx9q9nzZC4KPfM85og1ybjTx5'
* PINECONE_ENV = 'us-east-1'
* index_name = "boazpubmed"
* dimension = 1536
* metadata : [ID, paper_title, page_content, year]
* page_content : title + abstract
* 전처리 후 총 26239개 업로드 완.

In [14]:
import os
import pandas as pd
from langchain_core.documents import Document
from langchain_community.embeddings import SentenceTransformerEmbeddings
from pinecone import Pinecone, ServerlessSpec

In [15]:
from dotenv import load_dotenv

load_dotenv()

True

- title, year가 NAN인 행들 12개는 직접 채워줬고,
- abstract가 NAN인 1700여개는 제거함.

In [ ]:
import pandas as pd
from langchain_core.documents import Document

excel_path = r"C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\마취관련_키워드_pubmed_abstract_extraction_sample.xlsx"
df = pd.read_excel(excel_path)
df = df.dropna(subset=['article_abstract'])

In [ ]:
df.head()

,Unnamed: 0,article_title,article_abstract,year,article_url
0,0,3D Imaging Reveals Complex Microvascular Remod...,BACKGROUND: Pathogenic concepts of right ventr...,2024,https://ncbi.nlm.nih.gov/pubmed/38770652
1,1,Neonatal isoflurane exposure disturbs granule ...,It has been reported that neonatal isoflurane ...,2022,https://ncbi.nlm.nih.gov/pubmed/35173111
2,2,Differential Effects of a Behavioral Treatment...,The purpose of this study was to examine poten...,2023,https://ncbi.nlm.nih.gov/pubmed/37142899
3,3,Risk factors and prevention of choking.,Choking (or foreign body airway obstruction) i...,2023,https://ncbi.nlm.nih.gov/pubmed/37905785
4,4,Complications and adverse events in lymphadene...,BACKGROUND: Inguinal lymph node dissection pla...,2024,https://ncbi.nlm.nih.gov/pubmed/38987232


In [ ]:
num_nan_rows = df.isnull().any(axis=1).sum()
print(f"NaN이 하나라도 포함된 행의 개수: {num_nan_rows}")

NaN이 하나라도 포함된 행의 개수: 0


In [ ]:
documents = []
for i, row in df.iterrows():
    title = str(row.get("article_title", "")).strip()
    abstract = str(row.get("article_abstract", "")).strip()
    year = row.get("year", "")

    # title + abstract 합치기(선택)
    full_text = f"{title}\n\n{abstract}" if abstract else title

    metadata = {
        "paper_title": title,
        "year": year,
        "page_content": full_text
    }
    doc = Document(page_content=full_text, metadata=metadata)
    documents.append(doc)

print(f"✅ 엑셀 기반 문서 {len(documents)}개 생성 완료")

✅ 엑셀 기반 문서 26239개 생성 완료


In [ ]:
from pinecone import Pinecone, ServerlessSpec

# ✅ 환경변수에서 API 키 불러오기
api_key = os.environ.get("PINECONE_API_KEY")
if not api_key:
    raise ValueError("❌ PINECONE_API_KEY 환경변수가 설정되지 않았습니다.")

# ✅ Pinecone 클라이언트 생성
pc = Pinecone(api_key=api_key)

# ✅ 인덱스 이름 및 설정
index_name = "boazpubmed"
dimension = 1536  # 예: OpenAI embedding 모델 "text-embedding-ada-002" 사용 시

# ✅ 인덱스 생성 (이미 존재하는 경우 생략)
if index_name not in [i.name for i in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print(f"✅ 인덱스 '{index_name}' 생성 완료!")
else:
    print(f"✅ 인덱스 '{index_name}' 이미 존재합니다.")


✅ 인덱스 'boazpubmed' 이미 존재합니다.


In [ ]:
import os
from openai import OpenAI
from uuid import uuid4
from pinecone import Pinecone
from tqdm import tqdm

#  API 키 불러오기
openai_api_key = os.environ["OPENAI_API_KEY"]
pinecone_api_key = os.environ["PINECONE_API_KEY"]

#  Pinecone 및 OpenAI 클라이언트 생성
pc = Pinecone(api_key=pinecone_api_key)
client = OpenAI(api_key=openai_api_key)
index = pc.Index("boazpubmed")  # <- 인덱스 이름 맞게 변경!

# 메타데이터 정리 함수
def clean_metadata(meta: dict) -> dict:
    cleaned = {}
    for k, v in meta.items():
        if v is None:
            continue
        if isinstance(v, (str, int, float, bool)):
            cleaned[k] = v
        elif isinstance(v, list) and all(isinstance(i, str) for i in v):
            cleaned[k] = v
        else:
            cleaned[k] = str(v)
    return cleaned


In [ ]:
# 업로드 루프
batch_size = 100
vectors = []

for i, doc in enumerate(tqdm(documents)):
    try:
        # 임베딩 생성
        response = client.embeddings.create(
            model="text-embedding-ada-002",
            input=doc.page_content
        )
        embedding = response.data[0].embedding

        metadata = clean_metadata(doc.metadata)

        vectors.append({
            "id": f"pubmed_{i}", 
            "values": embedding,
            "metadata": metadata
        })

        if len(vectors) == batch_size:
            index.upsert(vectors)
            vectors = []

    except Exception as e:
        print(f"❌ 에러 발생 (i={i}): {e}")

if vectors:
    index.upsert(vectors)
    print("남은 벡터 업로드 완료")


print("임베딩 벡터 저장 완료!")

* 중간에 멈춘 곳부터 재업로드

In [ ]:
# Pinecone에서 pubmed_0 ~ pubmed_n의 존재 여부를 이진 탐색식으로 빠르게 체크
def find_last_index(index, max_search=30000):
    left, right = 0, max_search
    last_found = -1
    while left <= right:
        mid = (left + right) // 2
        res = index.fetch(ids=[f"pubmed_{mid}"])
        if f"pubmed_{mid}" in res["vectors"]:
            last_found = mid
            left = mid + 1
        else:
            right = mid - 1
    return last_found

last_idx = find_last_index(index)
print(f"현재 Pinecone에 저장된 마지막 pubmed id: pubmed_{last_idx}")
start_idx = last_idx + 1
print(f"{start_idx}번째부터 다시 업로드를 시작합니다.")

In [13]:
batch_size = 100
vectors = []

for i in tqdm(range(start_idx, len(documents))):
    doc = documents[i]
    try:
        response = client.embeddings.create(
            model="text-embedding-ada-002",
            input=doc.page_content
        )
        embedding = response.data[0].embedding
        metadata = clean_metadata(doc.metadata)
        vectors.append({
            "id": f"pubmed_{i}",
            "values": embedding,
            "metadata": metadata
        })
    except Exception as e:
        print(f"❌ 에러 발생 (i={i}): {e}")

    if len(vectors) == batch_size:
        try:
            index.upsert(vectors)
            print(f"✅ {i+1}번째까지 업로드 완료 ({batch_size}개)")
            vectors = []
        except Exception as e:
            print(f"❌ 업로드 에러 (i={i}): {e}")

if vectors:
    try:
        index.upsert(vectors)
        print("✅ 마지막 배치 업로드 완료")
    except Exception as e:
        print(f"❌ 마지막 배치 업로드 에러: {e}")

print("전체 임베딩 벡터 업로드 시도 완료!")

 54%|█████▍    | 100/186 [01:10<11:09,  7.79s/it]

✅ 26153번째까지 업로드 완료 (100개)


100%|██████████| 186/186 [01:50<00:00,  1.69it/s]


✅ 마지막 배치 업로드 완료
전체 임베딩 벡터 업로드 시도 완료!
